# Calculating ROUGE scores
This notebook's purpose is to provide easy-to-install environment for calculating ROUGE scores on provided summaries.

## Prequisites
To calculate ROUGE score, one must connect to Google Drive and have these directories and files:
* directory `result/<model_name>` with zipped directory with summaries generated by the model after running `main.py` script and zipped directory with gold standard summaries.

## Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing ROUGE and Python wrapper - `pyrouge`

In [2]:
!git clone https://github.com/andersjo/pyrouge.git rouge

Cloning into 'rouge'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (393/393), 298.74 KiB | 11.49 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [3]:
!pip install pyrouge

     |████████████████████████████████| 60 kB 4.8 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191621 sha256=610d59b99c53dd90c9e6232ce9394e64b2f49a830df77f9d034f0e3ba2cab65e
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge


In [4]:
!pyrouge_set_rouge_path '/content/rouge/tools/ROUGE-1.5.5'

2022-05-10 17:08:29,950 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/rouge/tools/ROUGE-1.5.5.


In [5]:
%cd rouge/tools/ROUGE-1.5.5/data
!rm "WordNet-2.0.exc.db"
!perl ./WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db
!cpan install XML::DOM

/content/rouge/tools/ROUGE-1.5.5/data
Loading internal null logger. Install Log::Log4perl for logging messages

CPAN.pm requires configuration, but most of it can be done automatically.
If you answer 'no' below, you will enter an interactive dialog for each
configuration option instead.

Would you like to configure as much as possible automatically? [yes] yes
Fetching with HTTP::Tiny:
http://www.cpan.org/authors/01mailrc.txt.gz
Reading '/root/.cpan/sources/authors/01mailrc.txt.gz'
............................................................................DONE
Fetching with HTTP::Tiny:
http://www.cpan.org/modules/02packages.details.txt.gz
Reading '/root/.cpan/sources/modules/02packages.details.txt.gz'
  Database was generated on Tue, 10 May 2022 16:17:03 GMT
  HTTP::Date not available
..............
  New CPAN.pm version (v2.34) available.
  [Currently running version is v2.18]
  You might want to try
    install CPAN
    reload cpan
  to both upgrade CPAN.pm and run the new version wi

In [6]:
from pyrouge import Rouge155
import csv
import os

## Testing on MatchSum summaries

In [ ]:
r = Rouge155()
r.system_dir = '/content/drive/MyDrive/result/MatchSum_cnndm_bert.ckpt/dec/'
r.model_dir = '/content/drive/MyDrive/result/MatchSum_cnndm_bert.ckpt/ref/'
r.system_filename_pattern = '(\d+).dec'
r.model_filename_pattern = '#ID#.ref'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

In [ ]:
with open("/content/drive/MyDrive/result/MatchSum_cnndm_bert.ckpt/rouge.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["metric", "value"])
    writer.writeheader()
    for metric, value in output_dict.items():
        writer.writerow({"metric": metric, "value":value})

# Our models results

In [7]:
%env SUMMARY_DIR=/content/drive/MyDrive/result/pso_first_model/
%env SUMMARY_FILE=summary.zip
%env TARGET_DIR=/content/drive/MyDrive/result/
%env TARGET_FILE=targets.zip

env: SUMMARY_DIR=/content/drive/MyDrive/result/pso_first_model/
env: SUMMARY_FILE=summary.zip
env: TARGET_DIR=/content/drive/MyDrive/result/
env: TARGET_FILE=targets.zip


In [8]:
! unzip $SUMMARY_DIR$SUMMARY_FILE -d $SUMMARY_DIR

Archive:  /content/drive/MyDrive/result/pso_first_model/summary.zip
replace /content/drive/MyDrive/result/pso_first_model/summary/00000.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [9]:
! unzip $TARGET_DIR$TARGET_FILE -d $TARGET_DIR

Archive:  /content/drive/MyDrive/result/targets.zip
replace /content/drive/MyDrive/result/targets/00000.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [10]:
r = Rouge155()
r.system_dir = os.environ["SUMMARY_DIR"] + os.environ["SUMMARY_FILE"].split(".")[0]
r.model_dir = os.environ["TARGET_DIR"] + os.environ["TARGET_FILE"].split(".")[0]
r.system_filename_pattern = '(\d+).txt'
r.model_filename_pattern = '#ID#.txt'

output = r.convert_and_evaluate()
output_dict = r.output_to_dict(output)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
2022-05-10 17:22:07,067 [MainThread  ] [INFO ]  Processing 04876.txt.
2022-05-10 17:22:07,072 [MainThread  ] [INFO ]  Processing 04878.txt.
2022-05-10 17:22:07,077 [MainThread  ] [INFO ]  Processing 04881.txt.
2022-05-10 17:22:07,084 [MainThread  ] [INFO ]  Processing 04880.txt.
2022-05-10 17:22:07,091 [MainThread  ] [INFO ]  Processing 04879.txt.
2022-05-10 17:22:07,096 [MainThread  ] [INFO ]  Processing 04883.txt.
2022-05-10 17:22:07,102 [MainThread  ] [INFO ]  Processing 04882.txt.
2022-05-10 17:22:07,109 [MainThread  ] [INFO ]  Processing 04884.txt.
2022-05-10 17:22:07,115 [MainThread  ] [INFO ]  Processing 04885.txt.
2022-05-10 17:22:07,121 [MainThread  ] [INFO ]  Processing 04886.txt.
2022-05-10 17:22:07,126 [MainThread  ] [INFO ]  Processing 04888.txt.
2022-05-10 17:22:07,131 [MainThread  ] [INFO ]  Processing 04887.txt.
2022-05-10 17:22:07,136 [MainThread  ] [INFO ]  Processing 04889.txt.
2022-05-10 17:22:07,143 [

In [11]:
print(output)

---------------------------------------------
1 ROUGE-1 Average_R: 0.49291 (95%-conf.int. 0.49023 - 0.49575)
1 ROUGE-1 Average_P: 0.28480 (95%-conf.int. 0.28253 - 0.28719)
1 ROUGE-1 Average_F: 0.34204 (95%-conf.int. 0.33993 - 0.34433)
---------------------------------------------
1 ROUGE-2 Average_R: 0.18437 (95%-conf.int. 0.18194 - 0.18683)
1 ROUGE-2 Average_P: 0.10853 (95%-conf.int. 0.10692 - 0.11040)
1 ROUGE-2 Average_F: 0.12963 (95%-conf.int. 0.12794 - 0.13162)
---------------------------------------------
1 ROUGE-3 Average_R: 0.09730 (95%-conf.int. 0.09544 - 0.09935)
1 ROUGE-3 Average_P: 0.05871 (95%-conf.int. 0.05739 - 0.06033)
1 ROUGE-3 Average_F: 0.06942 (95%-conf.int. 0.06797 - 0.07111)
---------------------------------------------
1 ROUGE-4 Average_R: 0.06228 (95%-conf.int. 0.06063 - 0.06406)
1 ROUGE-4 Average_P: 0.03818 (95%-conf.int. 0.03696 - 0.03958)
1 ROUGE-4 Average_F: 0.04475 (95%-conf.int. 0.04346 - 0.04624)
---------------------------------------------
1 ROUGE-L Aver

In [12]:
with open(os.environ["SUMMARY_DIR"] + "rouge.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["metric", "value"])
    writer.writeheader()
    for metric, value in output_dict.items():
        writer.writerow({"metric": metric, "value":value})